In [31]:
import random
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
import seaborn as sns
import numpy as np
import pickle
from joblib import Parallel, delayed
from math import log2, ceil
from proglearn.progressive_learner import ProgressiveLearner, LifelongClassificationForest
from proglearn.deciders import SimpleArgmaxAverage, KNNClassificationDecider
from proglearn.transformers import TreeClassificationTransformer, NeuralClassificationTransformer
from proglearn.voters import TreeClassificationVoter, KNNClassificationVoter
from proglearn.sims import generate_gaussian_parity

ImportError: cannot import name 'LifelongClassificationForest' from 'proglearn.progressive_learner' (C:\Users\shizh\Anaconda3\envs\proglearn\lib\site-packages\proglearn-0.0.3-py3.7.egg\proglearn\progressive_learner.py)

In [32]:
X_xor, y_xor = generate_gaussian_parity(n_samples=1000)
X_xor_tmp = X_xor

indices = np.where((y_xor==0))[0]
to_replace = np.random.permutation(indices)[:int(indices.size * 0.9)]

X_xor = np.delete(X_xor, to_replace, axis=0)
y_xor = np.delete(y_xor, to_replace, axis=0)

temp = np.where(X_xor == X_xor_tmp[~to_replace][0,:])[0][0]
print(X_xor[temp,:], X_xor_tmp[~to_replace][0,:])
print(X_xor.dtype, X_xor_tmp.dtype)

colors = sns.color_palette('Dark2', n_colors=2)

fig, ax = plt.subplots(1,1, figsize=(8,8))

clr = [colors[i] for i in y_xor]
ax.scatter(X_xor[:, 0], X_xor[:, 1], c=clr, s=50)
# ax.set_xticks([])
# ax.set_yticks([])
ax.set_title('Gaussian XOR', fontsize=30)
# ax.axis('off')

IndexError: index 0 is out of bounds for axis 0 with size 0

In [33]:
def experiment_BTE(samples_task_1, samples_task_2, n_test, n_trees, max_depth, corr,
                   means_1, means_2,k_neighbors, acorn=None):
    
    # See if the sample sizes for both training sets are given.
    if samples_task_1 == 0 and samples_task_2 == 0:
        raise ValueError("Wake up and provide samples to train!!!")

    # If acorn is specified, set random seed to it.
    if acorn != None:
        np.random.seed(acorn)

    # Initialize array for storing errors, task 1, and task 2.
    errors = np.zeros((len(means_2)+1), dtype=float)

    # Initialize the transformer type and arguments.
    default_transformer_class = TreeClassificationTransformer
    default_transformer_kwargs = {"kwargs": {"max_depth": max_depth}}

    # Initialize the voter type and arguments.
    default_voter_class = TreeClassificationVoter
    default_voter_kwargs = {}

    # Initialize the decider type and arguments.
    default_decider_class = KNNClassificationDecider
    default_decider_kwargs = {"classes" : np.arange(2), "k": k_neighbors}

    # Initialize the progressive learner using the transformer, voter and decider classes.
    progressive_learner = LifelongClassificationForest(default_transformer_class = default_transformer_class, 
                                                       default_transformer_kwargs = default_transformer_kwargs,
                                                       default_voter_class = default_voter_class,
                                                       default_voter_kwargs = default_voter_kwargs,
                                                       default_decider_class = default_decider_class,
                                                       default_decider_kwargs = default_decider_kwargs)
    
    # Create the datasets with the Gaussian mean for task 1.
    X_task1, y_task1 = generate_gaussian_parity(samples_task_1)
    indices = np.where((y_task1==0))[0]
    to_replace = np.random.permutation(indices)[:int(indices.size * 0.9)]
    X_task1 = np.delete(X_task1, to_replace, axis=0)
    y_task1 = np.delete(y_task1, to_replace, axis=0)
    X_task1 = X_task1 - means_1

    test_task1, test_label_task1 = generate_gaussian_parity(n_test)
    indices = np.where((test_label_task1==0))[0]
    to_replace = np.random.permutation(indices)[:int(indices.size * 0.9)]
    test_task1 = np.delete(test_task1, to_replace, axis=0)
    test_label_task1 = np.delete(test_label_task1, to_replace, axis=0)
    test_task1 = test_task1 - means_1

    # Add a task for the task 1, predict the probabilities and add the MSE to the error array.
    progressive_learner.add_task(X_task1, y_task1)
    predicted_Z1 = progressive_learner.predict(test_task1, task_id=0)
    errors[0] = np.mean(predicted_Z1 == test_label_task1)
   
    # Then, add the transformer trained on task 2, predict and add the MSE to the error array.
    for i in range(len(means_2)):
        X_task2,y_task2 = generate_gaussian_parity(samples_task_2)
        indices = np.where((y_task2==0))[0]
        to_replace = np.random.permutation(indices)[:int(indices.size * 0.9)]
        X_task2 = np.delete(X_task2, to_replace, axis=0)
        y_task2 = np.delete(y_task2, to_replace, axis=0)
        
        X_task2 = X_task2 - means_2[i]
        
        progressive_learner.add_transformer(X_task2, y_task2)
        predicted_transformer_Z1 = progressive_learner.predict(test_task1, task_id=0)
        errors[i+1] = np.mean(predicted_transformer_Z1 == test_label_task1)
        
    return errors

In [34]:
# Set up the tree parameters.
mc_rep = 250
n_test = 100
n_trees = 10
n_sample_size = 100
max_depth = 10

# Set up the data parameters.
corr = 0
means_1 = [(12,12)]
means_2 = [(-4,-4),(-3,-3),(-2,-2),(-1,-1),(0,0),(1,1),(2,2),(3,3),(4,4),
          (5,5),(6,6),(7,7),(8,8),(9,9),(10,10),(11,11),(12,12)]
means_2.reverse()
k_neighbors = [3,5,9,17,33]

# Set up sample sizes.
samples_task_1 = 100
samples_task_2 = 100

In [35]:
# Initiate error arrays
mean_error_bte = np.zeros((len(k_neighbors), len(means_1) + len(means_2)))
std_error_bte = np.zeros((len(k_neighbors), len(means_1) + len(means_2)))

# Initiate transfer efficiency arrays
mean_bte = np.zeros((len(k_neighbors), len(means_1) + len(means_2)))

for i, n1 in enumerate(k_neighbors):
    # Create the error.
    error = np.array(
        Parallel(n_jobs=-1, verbose=1)(
            delayed(experiment_BTE)(
                samples_task_1, samples_task_2, n_test, n_trees, 
                max_depth, corr, means_1[0], means_2, n1) for _ in range(mc_rep)
            )
        )

    mean_error_bte[i] = np.mean(error, axis=0)
    mean_bte[i] = mean_error_bte[i,0] / mean_error_bte[i]

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


NameError: name 'LifelongClassificationForest' is not defined

In [ ]:
# Create empty Euclidean distance.
ran = np.arange((mean_bte).shape[1])
labels = ['k=3','k=5','k=9','k=17','k=33']

# Plot the BTEs
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
ax.plot(ran, mean_bte[0], ls="-", lw=3, marker='o')
ax.plot(ran, mean_bte[1], ls="-", lw=3, marker='o')
ax.plot(ran, mean_bte[2], ls="-", lw=3, marker='o')
ax.plot(ran, mean_bte[3], ls="-", lw=3, marker='o')
ax.plot(ran, mean_bte[4], ls="-", lw=3, marker='o')

ax.set_ylabel("Backward Transfer Efficiency wrt Task 1", fontsize=20)
ax.legend(labels, loc="upper left", fontsize=16, frameon=True)
ax.set_xlabel("Task Number", fontsize=20)
ax.axvline(x=0, c="gray", linewidth=1.5, linestyle="dashed")
ax.axhline(y=1, c="gray", linewidth=1.5, linestyle="dashed")
ax.set_xticks(ran)
ax.set_title('Setting 3', fontsize=20)

right_side = ax.spines["right"]
right_side.set_visible(False)
top_side = ax.spines["top"]
top_side.set_visible(False)
plt.tight_layout()